In [1]:
import pandas as pd
import os
import time
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.prompts import PromptTemplate

# ================= 配置区域 =================
# 🔴 请替换你的 Key
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

# ================= 你的核心 Agent 类 (Robust Engine) =================
class RobustPandasQueryEngine(PandasQueryEngine):
    def _query(self, query_bundle):
        # 第一次尝试
        response = super()._query(query_bundle)
        response_str = str(response)
        
        # 检查错误
        if "error running" in response_str or "Error message:" in response_str:
            print(f"    [Agent] ⚠️ 初次运行失败，触发纠错...")
            
            current_columns = list(self._df.columns)
            
            # V6 强力 Prompt
            retry_prompt_str = (
                "你是一个 Python 表达式生成器。用户的代码报错了。\n"
                "用户问题: {query}\n"
                "报错信息: {error_msg}\n\n"
                "**数据表当前列名**: {columns_info}\n"
                "**任务**：分析报错原因（可能是符号、日期格式或单位），生成一行清洗并计算的代码。\n\n"
                "**针对不同场景的清洗指南**：\n"
                "1. 货币/符号: .str.replace(r'[$,kgA-Za-z]', '', regex=True).astype(float)\n"
                "2. 日期问题: pd.to_datetime(df['列名'], errors='coerce').dt.month\n"
                "3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)\n\n"
                "**格式约束**：\n"
                "1. 禁止 import。\n"
                "2. 必须是一行 Python 代码，直接使用 df 变量。\n"
                "3. 严禁使用占位符。\n\n"
                "请直接输出代码："
            )
            
            retry_prompt = PromptTemplate(retry_prompt_str).format(
                query=query_bundle.query_str,
                error_msg=response_str,
                columns_info=str(current_columns) 
            )
            
            # 替换 Prompt 并重试
            original_prompt = self._pandas_prompt
            self._pandas_prompt = PromptTemplate(retry_prompt_str)
            
            try:
                retry_response = super()._query(query_bundle)
                if "Error message:" not in str(retry_response):
                    print("    [Agent] ✅ 纠错成功！")
                    self._pandas_prompt = original_prompt
                    return retry_response
                else:
                    print("    [Agent] ❌ 纠错后依然失败。")
            except Exception:
                pass
            
            self._pandas_prompt = original_prompt
            return response # 返回包含错误信息的原始响应
        
        else:
            return response

# ================= 批量评测逻辑 =================

def run_experiment():
    # 定义测试任务
    # (文件名, 用户问题, 这里的陷阱列名用于验证)
    tasks = [
        {
            "file": "bench_currency.csv",
            "question": "请计算 amount 列的平均值",
            "name": "测试 1: 货币符号"
        },
        {
            "file": "bench_dates.csv",
            "question": "请告诉我 transaction_id 103 是在哪个月份 (month) 发生的？(请提取月份数字)",
            "name": "测试 2: 日期解析"
        },
        {
            "file": "bench_units.csv",
            "question": "请计算 weight_val 列的总和 (sum)",
            "name": "测试 3: 物理单位"
        }
    ]
    
    results = []

    print("\n🚀 开始批量评测 (Batch Evaluation)...\n")
    
    for task in tasks:
        print(f"正在执行: {task['name']} ...")
        
        # 1. 加载数据
        if not os.path.exists(task['file']):
            print(f"❌ 找不到文件 {task['file']}，请先运行生成脚本！")
            continue
            
        df = pd.read_csv(task['file'])
        
        # 2. 初始化你的 Agent
        engine = RobustPandasQueryEngine(df=df, verbose=False) # 关闭啰嗦的 verbose
        
        # 3. 记录时间与结果
        start_time = time.time()
        try:
            response = engine.query(task['question'])
            final_output = str(response)
        except Exception as e:
            final_output = f"Crash: {str(e)}"
        end_time = time.time()
        
        # 4. 判断是否成功 (简单的关键词判断)
        # 如果结果里还有 "Error" 或者 "Crash"，就算失败
        is_success = "Error" not in final_output and "Crash" not in final_output
        
        results.append({
            "Task": task['name'],
            "Success": "✅ Pass" if is_success else "❌ Fail",
            "Output": final_output[:100] + "..." if len(final_output) > 100 else final_output,
            "Time": f"{end_time - start_time:.2f}s"
        })
        print(f"   结果: {'✅ Pass' if is_success else '❌ Fail'}\n")

    # ================= 输出最终报表 =================
    print("\n" + "="*50)
    print("📝 最终实验报告 (Table 1)")
    print("="*50)
    result_df = pd.DataFrame(results)
    print(result_df)
    print("="*50)

if __name__ == "__main__":
    run_experiment()

<>:45: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\d'
C:\Users\world\AppData\Local\Temp\ipykernel_14720\2752033783.py:45: SyntaxWarning: invalid escape sequence '\d'
  "3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)\n\n"



🚀 开始批量评测 (Batch Evaluation)...

正在执行: 测试 1: 货币符号 ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


<unknown>:1: SyntaxWarning: invalid escape sequence '\$'


   结果: ✅ Pass

正在执行: 测试 2: 日期解析 ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
   结果: ✅ Pass

正在执行: 测试 3: 物理单位 ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
   结果: ✅ Pass


📝 最终实验报告 (Table 1)
         Task Success         Output   Time
0  测试 1: 货币符号  ✅ Pass         2000.0  3.82s
1  测试 2: 日期解析  ✅ Pass  March 5, 2023  2.39s
2  测试 3: 物理单位  ✅ Pass          180.0  1.91s


In [2]:
import pandas as pd
import os
import time
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.prompts import PromptTemplate

# ================= 配置区域 =================
# 🔴 请替换你的 Key
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

# ================= 你的核心 Agent 类 (Robust Engine) =================
class RobustPandasQueryEngine(PandasQueryEngine):
    def _query(self, query_bundle):
        # 第一次尝试
        response = super()._query(query_bundle)
        response_str = str(response)
        
        # 检查错误
        if "error running" in response_str or "Error message:" in response_str:
            print(f"    [Agent] ⚠️ 初次运行失败，触发纠错...")
            
            current_columns = list(self._df.columns)
            
            # V6 强力 Prompt
            retry_prompt_str = (
                "你是一个 Python 表达式生成器。用户的代码报错了。\n"
                "用户问题: {query}\n"
                "报错信息: {error_msg}\n\n"
                "**数据表当前列名**: {columns_info}\n"
                "**任务**：分析报错原因（可能是符号、日期格式或单位），生成一行清洗并计算的代码。\n\n"
                "**针对不同场景的清洗指南**：\n"
                "1. 货币/符号: .str.replace(r'[$,kgA-Za-z]', '', regex=True).astype(float)\n"
                "2. 日期问题: pd.to_datetime(df['列名'], errors='coerce').dt.month\n"
                "3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)\n\n"
                "**格式约束**：\n"
                "1. 禁止 import。\n"
                "2. 必须是一行 Python 代码，直接使用 df 变量。\n"
                "3. 严禁使用占位符。\n\n"
                "请直接输出代码："
            )
            
            retry_prompt = PromptTemplate(retry_prompt_str).format(
                query=query_bundle.query_str,
                error_msg=response_str,
                columns_info=str(current_columns) 
            )
            
            # 替换 Prompt 并重试
            original_prompt = self._pandas_prompt
            self._pandas_prompt = PromptTemplate(retry_prompt_str)
            
            try:
                retry_response = super()._query(query_bundle)
                if "Error message:" not in str(retry_response):
                    print("    [Agent] ✅ 纠错成功！")
                    self._pandas_prompt = original_prompt
                    return retry_response
                else:
                    print("    [Agent] ❌ 纠错后依然失败。")
            except Exception:
                pass
            
            self._pandas_prompt = original_prompt
            return response # 返回包含错误信息的原始响应
        
        else:
            return response

# ================= 批量评测逻辑 =================

def run_experiment():
    # 定义测试任务
    # (文件名, 用户问题, 这里的陷阱列名用于验证)
    # ... (前面的代码不变)

    print("\n🚀 开始批量评测真实数据 (WikiTableQuestions)...\n")
    
    # 1. 读取任务列表
    task_df = pd.read_csv("real_benchmark_tasks.csv")
    
    # 转换为字典列表
    tasks = task_df.to_dict('records') 
    # 现在的 tasks 里的 key 是: 'file', 'question', 'original_answer'
    
    results = []

    for i, task in enumerate(tasks):
        print(f"[{i+1}/{len(tasks)}] 正在执行: {task['question'][:50]}...")
        
        # ... (中间的执行逻辑不变) ...
    
    #results = []

    print("\n🚀 开始批量评测 (Batch Evaluation)...\n")
    
    for task in tasks:
        print(f"正在执行: {task['name']} ...")
        
        # 1. 加载数据
        if not os.path.exists(task['file']):
            print(f"❌ 找不到文件 {task['file']}，请先运行生成脚本！")
            continue
            
        df = pd.read_csv(task['file'])
        
        # 2. 初始化你的 Agent
        engine = RobustPandasQueryEngine(df=df, verbose=False) # 关闭啰嗦的 verbose
        
        # 3. 记录时间与结果
        start_time = time.time()
        try:
            response = engine.query(task['question'])
            final_output = str(response)
        except Exception as e:
            final_output = f"Crash: {str(e)}"
        end_time = time.time()
        
        # 4. 判断是否成功 (简单的关键词判断)
        # 如果结果里还有 "Error" 或者 "Crash"，就算失败
        is_success = "Error" not in final_output and "Crash" not in final_output
        
        results.append({
            "Task": task['name'],
            "Success": "✅ Pass" if is_success else "❌ Fail",
            "Output": final_output[:100] + "..." if len(final_output) > 100 else final_output,
            "Time": f"{end_time - start_time:.2f}s"
        })
        print(f"   结果: {'✅ Pass' if is_success else '❌ Fail'}\n")

    # ================= 输出最终报表 =================
    print("\n" + "="*50)
    print("📝 最终实验报告 (Table 1)")
    print("="*50)
    result_df = pd.DataFrame(results)
    print(result_df)
    print("="*50)

if __name__ == "__main__":
    run_experiment()

<>:45: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\d'
C:\Users\world\AppData\Local\Temp\ipykernel_14720\2695753041.py:45: SyntaxWarning: invalid escape sequence '\d'
  "3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)\n\n"



🚀 开始批量评测真实数据 (WikiTableQuestions)...

[1/30] 正在执行: in what year was the least money won?...
[2/30] 正在执行: how many years were there more than 6 matches?...
[3/30] 正在执行: which tournaments had a margin victory of at most ...
[4/30] 正在执行: what is the total number of pasarams?...
[5/30] 正在执行: which aircraft came in service first?...
[6/30] 正在执行: what was the rams' largest margin of victory durin...
[7/30] 正在执行: after 1900 how many entries are there?...
[8/30] 正在执行: how many of the five largest flood on the river tr...
[9/30] 正在执行: what is the total amount of time it took the compe...
[10/30] 正在执行: which tonga is the only one to feature chicken wit...
[11/30] 正在执行: what was the first team played against in the mont...
[12/30] 正在执行: what model has the top mpg (us gallons)?...
[13/30] 正在执行: what is the total number of championships for tom ...
[14/30] 正在执行: which country did it take the longest to finish?...
[15/30] 正在执行: name of the festivals that had at least 2 dates....
[16/30] 正在执行: how m

C:\Users\world\AppData\Local\Temp\ipykernel_14720\2695753041.py:45: SyntaxWarning: invalid escape sequence '\d'
  "3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)\n\n"


KeyError: 'name'

In [3]:
import pandas as pd
import os
import time
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.prompts import PromptTemplate

# ================= 配置区域 =================
# 🔴 请替换你的 Key
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

# ================= 你的核心 Agent 类 (Robust Engine V6) =================
class RobustPandasQueryEngine(PandasQueryEngine):
    def _query(self, query_bundle):
        # 第一次尝试
        response = super()._query(query_bundle)
        response_str = str(response)
        
        # 检查错误
        if "error running" in response_str or "Error message:" in response_str:
            # print(f"    [Agent] ⚠️ 初次运行失败，触发纠错...") # 为了报表整洁，注释掉这行
            
            current_columns = list(self._df.columns)
            
            # V6 强力 Prompt
            retry_prompt_str = (
                "你是一个 Python 表达式生成器。用户的代码报错了。\n"
                "用户问题: {query}\n"
                "报错信息: {error_msg}\n\n"
                "**数据表当前列名**: {columns_info}\n"
                "**任务**：分析报错原因，生成一行清洗并计算的代码。\n\n"
                "**针对不同场景的清洗指南**：\n"
                r"1. 货币/符号: .str.replace(r'[$,kgA-Za-z]', '', regex=True).astype(float)" + "\n"
                r"2. 日期问题: pd.to_datetime(df['列名'], errors='coerce').dt.month" + "\n"
                r"3. 混合单位: .str.extract(r'(\d+)')[0].astype(float)" + "\n\n"
                "**格式约束**：\n"
                "1. 禁止 import。\n"
                "2. 必须是一行 Python 代码，直接使用 df 变量。\n"
                "3. 严禁使用占位符。\n\n"
                "请直接输出代码："
            )
            
            retry_prompt = PromptTemplate(retry_prompt_str).format(
                query=query_bundle.query_str,
                error_msg=response_str,
                columns_info=str(current_columns) 
            )
            
            # 替换 Prompt 并重试
            original_prompt = self._pandas_prompt
            self._pandas_prompt = PromptTemplate(retry_prompt_str)
            
            try:
                retry_response = super()._query(query_bundle)
                # 恢复 Prompt
                self._pandas_prompt = original_prompt
                return retry_response
            except Exception:
                pass
            
            self._pandas_prompt = original_prompt
            return response 
        else:
            return response

# ================= 批量评测逻辑 =================

def run_experiment():
    print("\n🚀 开始批量评测真实数据 (WikiTableQuestions)...")
    print("----------------------------------------------------------------")
    
    # 1. 读取任务列表
    if not os.path.exists("real_benchmark_tasks.csv"):
        print("❌ 错误: 找不到 real_benchmark_tasks.csv，请先运行 load_wtq_manual.py")
        return

    task_df = pd.read_csv("real_benchmark_tasks.csv")
    
    # 转换为字典列表
    tasks = task_df.to_dict('records') 
    
    results = []
    
    # 2. 遍历任务
    for i, task in enumerate(tasks):
        # 截取过长的问题用于显示
        short_question = (task['question'][:40] + '...') if len(task['question']) > 40 else task['question']
        print(f"[{i+1}/{len(tasks)}] 问题: {short_question}")
        
        # 检查文件是否存在
        if not os.path.exists(task['file']):
            print(f"    ❌ 跳过: 文件不存在 {task['file']}")
            continue
            
        try:
            # 加载 CSV
            df = pd.read_csv(task['file'])
            
            # 初始化 Agent (关闭 verbose 以保持控制台清爽)
            engine = RobustPandasQueryEngine(df=df, verbose=False)
            
            start_time = time.time()
            
            # === 核心查询 ===
            response = engine.query(task['question'])
            
            end_time = time.time()
            final_output = str(response)
            
            # 判断是否崩溃 (Robustness Check)
            is_crash = "Error" in final_output or "Crash" in final_output
            status = "❌ CRASH" if is_crash else "✅ SUCCESS"
            
            # 打印简报
            print(f"    状态: {status} | 耗时: {end_time - start_time:.2f}s")
            print(f"    预测值: {final_output}")
            print(f"    真实值: {task['original_answer']}")
            print("-" * 30)

            results.append({
                "ID": task['id'],
                "Question": short_question,
                "Status": status,
                "Prediction": final_output,
                "Ground Truth": task['original_answer'],
                "Time": f"{end_time - start_time:.2f}s"
            })
            
        except Exception as e:
            print(f"    ❌ 严重系统错误: {e}")
            results.append({
                "ID": task['id'],
                "Question": short_question,
                "Status": "❌ SYSTEM ERROR",
                "Prediction": str(e),
                "Ground Truth": task['original_answer'],
                "Time": "0.00s"
            })

    # ================= 输出最终报表 =================
    print("\n" + "="*60)
    print("📝 最终实验报告 (WikiTableQuestions Subset)")
    print("="*60)
    
    final_df = pd.DataFrame(results)
    
    # 计算鲁棒性成功率 (非 Crash 的比例)
    success_count = len(final_df[final_df["Status"] == "✅ SUCCESS"])
    total_count = len(final_df)
    success_rate = (success_count / total_count) * 100
    
    print(final_df[["ID", "Question", "Status", "Prediction", "Ground Truth"]])
    print("="*60)
    print(f"🏆 鲁棒性成功率 (Robustness Score): {success_rate:.2f}% ({success_count}/{total_count})")
    print("="*60)
    
    # 保存结果到 Excel/CSV 方便写论文用
    final_df.to_csv("experiment_results_wtq.csv", index=False)
    print("结果已保存至 experiment_results_wtq.csv")

if __name__ == "__main__":
    run_experiment()


🚀 开始批量评测真实数据 (WikiTableQuestions)...
----------------------------------------------------------------
[1/30] 问题: in what year was the least money won?
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


<unknown>:1: SyntaxWarning: invalid escape sequence '\$'


    状态: ✅ SUCCESS | 耗时: 3.81s
    预测值: 1979
    真实值: 1979
------------------------------
[2/30] 问题: how many years were there more than 6 ma...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


Traceback (most recent call last):
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "d:\soft\Lib\site-packages\pandas\core\ops\common.py", line 76, in new_method
    return method(self, other)
           ^^^^^^^^^^^^^^^^^^^
  File "d:\soft\Lib\site-packages\pandas\core\arraylike.py", line 56, in __gt__
    return self._cmp_method(other, operator.gt)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


...

    状态: ✅ SUCCESS | 耗时: 2.76s
    预测值: 0    False
1    False
     ...  
4     True
5    False
Length: 6, dtype: bool
    真实值: 2
------------------------------
[9/30] 问题: what is the total amount of time it took...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS | 耗时: 3.02s
    预测值: 63.44
    真实值: 63.44
------------------------------
[10/30] 问题: which tonga is the only one to feature c...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS | 耗时: 2.46s
    预测值: Cupronickel
    真实值: 5 seniti
------------------------------
[11/30] 问题: what was the first team played against i...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/comple

Traceback (most recent call last):
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "d:\soft\Lib\site-packages\pandas\core\generic.py", line 6643, in astype
    new_data = self._mgr.astype(dtype=dtype, copy=copy, errors=errors)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\soft\Lib\site-packages\pandas\core\internals\managers.py", line 430, in astype
    return self.appl

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


Traceback (most recent call last):
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1
    df['{column}'] = pd.to_numeric(df['{column}'].str.replace('[$,]', '', regex=True), errors='coerce')
                   ^
SyntaxError: invalid syntax


    状态: ❌ CRASH | 耗时: 5.96s
    预测值: There was an error running the output as Python code. Error message: invalid syntax (<string>, line 1)
    真实值: 2008
------------------------------
[20/30] 问题: how many ships are listed as a steamer?
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS | 耗时: 2.11s
    预测值: 10
    真实值: 10
------------------------------
[21/30] 问题: only player to score more than 2 goals i...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS | 耗时: 2.82s
    预测值: Aloisi
    真实值: Aloisi
------------------------------
[22/30] 问题: what city has a radio station called the...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/c